## **Imports**


In [1]:
import os
import pprint
import asyncio
import aiohttp
import requests
import pandas as pd

C:\Users\mujta\AppData\Local\Temp\ipykernel_1376\456781877.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## **Loading Dataset**


In [2]:
restaurantsDf = pd.read_csv('./data/restaurants.csv', encoding='latin-1')

In [3]:
restaurantsDf = restaurantsDf.head(10000)

### **Start from columns where there is no DineEase Id**

In [4]:
if 'id' in restaurantsDf.columns:
    filteredDf = restaurantsDf[restaurantsDf['id'].isna()]
else:
    filteredDf = restaurantsDf

# Display the DataFrame
filteredDf.head(1)

,business_id,address,city,latitude,longitude,name,rating,review_count,categories
0,---kPU91CF4Lq2-WlRu9Lw,4903 State Rd 54,New Port Richey,28.217288,-82.733344,Frankie's Raw Bar,4.5,24,"Seafood, Latin American"


## **Workers**


### **Generate User**

In [5]:
users_backup_file = './data/backup/users_backup.csv'
users_dict = {}

if os.path.exists(users_backup_file):
    users_backup_df = pd.read_csv(users_backup_file)

    
    for index, row in users_backup_df.iterrows():
        user_details = {
            "id": row["id"],
            "email": row["email"],
            "token": row["token"]
        }
        users_dict[row["email"]] = user_details

In [6]:
restaurant_backup_file = './data/backup/restaurants_backup.csv'

if os.path.exists(restaurant_backup_file):
    restaurant_backup_df = pd.read_csv(restaurant_backup_file)
    restaurants_dict = dict(zip(restaurant_backup_df['restaurant_name'], restaurant_backup_df['owner']))
else:
    restaurants_dict = {}

In [7]:
# fetch random user
def fetch_random_user():
    response = requests.get('https://randomuser.me/api/')
    data = response.json()
    user_data = data['results'][0]
    user = {
        "firstName": user_data['name']['first'],
        "lastName": user_data['name']['last'],
        "email": user_data['email'],
        "password": "Mujtaba@123",
        "role": "Manager",
    }
    return user


# insert user in the DineEase database
async def insert_user(session, user):  
    async with session.post('http://dine-ease.dev/api/auth/register', json=user) as response:  
    
        if response.status == 201:
            new_user_data = await response.json()
            
            user['id'] = new_user_data['id']
            user['token'] = new_user_data['token']
        
            return user
        else:
            print(f"Error creating user. Status code: {response.status}")


# append user in dictionaries
async def create_user(session, restaurant_name):
    while True:
        user = fetch_random_user()
        
        if user is None:
            print("No user fetched. Retrying...")
            continue

        email = user['email']

        if users_dict.get(email):
            print(email, "is duplicated")
            continue
            
        else:
            new_user =  await insert_user(session, user)
            print(new_user)
            users_dict[email] = new_user
            restaurants_dict[restaurant_name] = email
            break

In [8]:
pprint.pprint(list(users_dict.values())[-2:])

[]


In [9]:
pprint.pprint(list(restaurants_dict.items())[-2:])

[]


### **Insert Restaurant**

In [10]:
async def process_restaurant(session, index, row):
    payload = {
        'name': row['name'],
        'taxId': str(index).zfill(13),
        'categories': [row['categories']],
        'address': row['address'],
        'location': {
            'coordinates': [row['longitude'], row['latitude']],
            'country': row['city'],
        },
    }
    
    # Check if restaurant_owner exists in users_dict
    if not restaurants_dict.get(row['name']):
        await create_user(session, row['name'])

    user = users_dict.get(restaurants_dict.get(row['name']))
    headers = {'Authorization': 'Bearer ' + user['token'], 'Content-Type': 'application/json'}

    async with session.post('http://dine-ease.dev/api/restaurant', json=payload, headers=headers) as response:
        if response.status == 201:
            data = await response.json()
            restaurantsDf.at[index, 'slug'] = data['slug']
            restaurantsDf.at[index, 'id'] = data['id']
            restaurantsDf.at[index, 'userId'] = user['id']
        else:
            pprint.pprint(f"Request failed for row {index + 1}. Status code: {response.status}")
            pprint.pprint(await response.text())

In [11]:
async def main():
    async with aiohttp.ClientSession() as session:
        tasks = [process_restaurant(session, index, row) for index, row in filteredDf.iloc[:500].iterrows()]
        await asyncio.gather(*tasks)

await main()

In [12]:
# # for index, row in restaurantsDf.iterrows():
# for index, row in filteredDf.iterrows():
    
#     payload = {
#         'name': row['name'],
#         'taxId': str(index).zfill(13),
#         'categories': [row['categories']],
#         'address': row['address'],
#         'location': {
#             'coordinates': [row['longitude'], row['latitude']],
#             'country': row['city'],
#         },
#     }
    
#     # Check if restaurant_owner exists in users_dict
#     if not restaurants_dict.get(row['name']):
#         create_user(row['name'])
    
#     user = users_dict.get(restaurants_dict.get(row['name']))
    
#     headers = { 'Authorization': 'Bearer ' + user['token'], 'Content-Type': 'application/json'}
#     response = requests.post('http://dine-ease.dev/api/restaurant', json=payload, headers=headers)
    
#     if response.status_code == 201:
#         data = response.json()
        
#         restaurantsDf.at[index, 'slug'] =  data['slug']
#         restaurantsDf.at[index, 'id'] = data['id']
#         restaurantsDf.at[index, 'userId'] = user['id']
        
#     else:
#         pprint.pprint(response.text)
#         print(f"Request failed for row {index + 1}. Status code: {response.status_code}")
#         break

### **Create backup incase of ERROR, hence code will continue from failure point**

In [16]:
# restaurantsDf.to_csv('./data/modified_restaurants.csv', index=False)

In [17]:
# # Create backup incase of ERROR, hence code will continue from failure point
# restuarant_backup_df = pd.DataFrame.from_dict({"restaurant_name": list(restaurants_dict.keys()), "owner": list(restaurants_dict.values())})
# restuarant_backup_df.to_csv('./data/backup/restaurants_backup.csv', index=False)

In [18]:
# users_backup_df = pd.DataFrame.from_dict({k: {"id": v["id"], "email": v["email"], "token": v["token"]} for k, v in users_dict.items()}, orient='index')
# users_backup_df = users_backup_df[['id', 'email', 'token']]
# users_backup_df.to_csv('./data/backup/users_backup.csv', index=False)